In [48]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

df_filtered = pd.read_csv(r'data\truck_loader_day.csv')
df_filtered['date']=pd.to_datetime(df_filtered['date'])
# Seleccionar las columnas relevantes
columnas_hdbscan = ['ton', 'n_shovel', 'truck_total_cycle', 'distance_empty', 'distance_full','trips']
df_hdbscan = df_filtered[columnas_hdbscan]


In [75]:
df_filtered.columns

Index(['Unnamed: 0', 'truck', 'loader', 'date', 'ton', 'n_shovel',
       'truck_total_cycle', 'loader_total_cycle', 'distance_empty',
       'distance_full', 'trips', 'cluster'],
      dtype='object')

In [49]:
df_hdbscan

,ton,n_shovel,truck_total_cycle,distance_empty,distance_full,trips
0,2110.433067,22.0,15169.0,37793.0,35087.0,7
1,3452.522876,35.0,19850.0,60107.0,53491.0,11
2,1351.985109,17.0,7588.0,20312.0,16106.0,4
3,1017.801653,12.0,6431.0,21696.0,17704.0,3
4,1551.033398,19.0,9878.0,28117.0,25855.0,5
...,...,...,...,...,...,...
59320,3413.993213,40.0,24039.0,58868.0,65874.0,11
59321,2169.387982,22.0,13015.0,34824.0,34734.0,7
59322,954.438321,11.0,5403.0,24203.0,13845.0,3
59323,9541.728748,107.0,54179.0,165151.0,141173.0,30


In [ ]:
# Estandarizar los datos
scaler = StandardScaler()
df_hdbscan_scaled = scaler.fit_transform(df_hdbscan)


array([[ 1.12184390e-01,  1.14581649e-01,  1.26245881e-01,
         1.48670420e-02,  2.93237908e-03,  1.87862610e-01],
       [ 9.88552442e-01,  9.27791232e-01,  5.68147251e-01,
         8.02579018e-01,  6.77383083e-01,  1.01924952e+00],
       [-3.83072749e-01, -1.98191267e-01, -5.89424763e-01,
        -6.02234018e-01, -6.92663622e-01, -4.35677570e-01],
       ...,
       [-6.42665886e-01, -5.73518767e-01, -7.95695750e-01,
        -4.64876874e-01, -7.75522405e-01, -6.43524296e-01],
       [ 4.96472859e+00,  5.43172123e+00,  3.80891463e+00,
         4.51076297e+00,  3.89066208e+00,  4.96833732e+00],
       [ 1.19712146e+00,  1.17800957e+00,  8.07176148e-01,
         9.80391388e-01,  7.88569968e-01,  1.22709624e+00]])

In [52]:
import hdbscan

# Configurar y ejecutar HDBSCAN
hdbscan_cluster = hdbscan.HDBSCAN(min_cluster_size=1000, min_samples=10)  # Ajusta min_cluster_size y min_samples según tus necesidades
clusters = hdbscan_cluster.fit_predict(df_hdbscan_scaled)

# Agregar los clusters al DataFrame original
df_filtered['cluster'] = clusters
df_filtered

,Unnamed: 0,truck,loader,date,ton,n_shovel,truck_total_cycle,loader_total_cycle,distance_empty,distance_full,trips,cluster
0,0,CAEX01,PH06,2023-01-01,2110.433067,22.0,15169.0,2278.0,37793.0,35087.0,7,7
1,1,CAEX01,PH06,2023-01-02,3452.522876,35.0,19850.0,3199.0,60107.0,53491.0,11,8
2,2,CAEX01,PH06,2023-01-03,1351.985109,17.0,7588.0,1755.0,20312.0,16106.0,4,5
3,3,CAEX01,PH06,2023-01-04,1017.801653,12.0,6431.0,777.0,21696.0,17704.0,3,3
4,4,CAEX01,PH06,2023-01-05,1551.033398,19.0,9878.0,1556.0,28117.0,25855.0,5,4
...,...,...,...,...,...,...,...,...,...,...,...,...
59320,59320,CAEX98,PH58,2024-05-20,3413.993213,40.0,24039.0,2785.0,58868.0,65874.0,11,-1
59321,59321,CAEX98,PH58,2024-05-21,2169.387982,22.0,13015.0,1954.0,34824.0,34734.0,7,7
59322,59322,CAEX98,PH58,2024-05-22,954.438321,11.0,5403.0,996.0,24203.0,13845.0,3,3
59323,59323,CAEX98,PH58,2024-05-23,9541.728748,107.0,54179.0,9622.0,165151.0,141173.0,30,-1


In [ ]:
numeric_cols = df_filtered.select_dtypes(include=['float64', 'int64']).columns
numeric_cols = [col for col in numeric_cols if col != 'cluster']
cluster_summary = df_filtered.groupby('cluster')[numeric_cols].agg(['mean', 'std']).reset_index()
cluster_summary


cluster    Unnamed: 0                        ton                n_shovel   
                    mean           std         mean          std       mean   
0       -1  31322.275895  16609.762086  4186.147280  1896.990043  43.901575  \
1        0  28959.309293  17459.948899   322.029216    17.376605   4.000000   
2        1  28749.575011  17349.850698   315.578134    16.599115   3.000000   
3        2  29106.817999  17347.956720   636.064994    25.821843   6.792841   
4        3  29123.675079  17154.422425   954.239744    33.142714  10.052426   
5        4  29246.535446  16960.193460  1589.783408    46.366528  16.534164   
6        5  29352.946140  17142.811517  1272.082172    39.695119  13.291823   
7        6  29384.865678  17286.476117  1907.594326    52.763059  19.777078   
8        7  30067.183753  17233.458490  2226.601261    58.656318  22.883467   
9        8  29312.366049  17143.030573  3363.611702   257.439655  34.224646   
10       9  29519.094682  17294.433298  2542.972585    64.118061  26.063554   
11      10  30127.312340  17015.769552  2860.650706    70.230595  29.182553   

              truck_total_cycle               loader_total_cycle                
          std              mean           std               mean          std   
0   19.747256      28289.935847  12770.080690        4005.319236  1850.863397  \
1    0.000000       2382.621491    753.077515         335.651985   102.804530   
2    0.000000       2388.959576    789.283722         294.647860   104.718728   
3    1.019330       4830.498562   1384.749375         622.887723   156.733770   
4    1.257233       7281.094637   2024.698599         923.030344   193.018882   
5    1.593684      12104.905844   3234.253427        1529.598095   252.638287   
6    1.405078       9706.673739   2646.399434        1227.930472   227.531824   
7    1.735711      14453.678114   3820.522351        1828.406828   279.156710   
8    1.779562      16659.726447   4455.234656        2113.966779   299.951804   
9    3.056389      21172.624137   5649.583110        3192.390403   446.239315   
10   1.770990      18743.467250   5060.960470        2413.333658   325.003881   
11   1.855885      20962.771489   5768.984892        2707.232766   340.780308   

   distance_empty               distance_full                    trips   
             mean           std          mean           std       mean   
0    74693.716563  34820.390445  70753.486110  33698.876715  13.161718  \
1     6331.680058   2261.340437   5726.798161   2169.934145   1.000000   
2     6796.900994   2329.902800   6167.351924   2212.463021   1.000000   
3    13245.541403   3981.276810  12235.912881   4135.578345   2.000000   
4    19943.847379   5644.860621  18621.691753   6105.850012   3.000000   
5    33420.597912   9059.515406  31151.904561   9947.596461   5.000000   
6    26651.447038   7342.730196  24910.343724   7978.806915   4.000000   
7    39792.499086  10825.183819  37254.006096  11694.084742   6.000000   
8    45933.228653  12439.856697  42669.189982  13256.823666   7.000000   
9    55970.028353  16634.594666  51919.463831  16567.765049  10.573610   
10   51486.220493  14511.250973  47847.148508  15123.294716   8.000000   
11   56999.756170  16486.221637  53362.054043  17237.273499   9.000000   

              
         std  
0   5.956321  
1   0.000000  
2   0.000000  
3   0.000000  
4   0.000000  
5   0.000000  
6   0.000000  
7   0.000000  
8   0.000000  
9   0.759149  
10  0.000000  
11  0.000000

In [74]:
# df_filtered[(df_filtered['cluster']==-1)& (df_filtered['ton']>13000)]
df_filtered[df_filtered['truck']=='CAEX66']['cluster'].value_counts()

cluster
-1     808
 3      85
 8      83
 5      81
 6      78
 4      77
 10     76
 9      75
 2      72
 7      69
 1      51
 0      24
Name: count, dtype: int64

In [58]:
# Revisar la distribución de camiones y palas por cluster
distribution_truck_loader = df_filtered.groupby(['cluster', 'truck', 'loader']).size().reset_index(name='count')
distribution_truck_loader.sort_values(by='count')


,cluster,truck,loader,count
301,0,CAEX61,PH06,1
2176,10,CAEX61,PH58,1
2175,10,CAEX61,PH55,1
2173,10,CAEX61,PH06,1
1427,6,CAEX61,PH06,1
...,...,...,...,...
43,-1,CAEX25,PH58,186
125,-1,CAEX66,PH06,189
101,-1,CAEX55,PH48,190
126,-1,CAEX66,PH48,242


In [59]:
df_filtered.to_csv(r'data\cluster.csv')

In [69]:
import plotly.subplots as sp
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Crear colores específicos para cada clúster
unique_clusters = cluster_summary.index
num_clusters = len(unique_clusters)
colors = [mcolors.rgb2hex(color) for color in plt.cm.tab10.colors][:num_clusters]

# Aseguramos el orden de los clusters y las métricas correspondientes
cluster_summary_reset = cluster_summary.reset_index()

# Crear una figura con subplots organizados en 2 filas x 3 columnas
fig = sp.make_subplots(rows=2, cols=3, subplot_titles=[
    "Tonelaje vs Número de Paladas",
    "Tonelaje vs Ciclo Total de Camión",
    "Tonelaje vs Distancia Recorrida Lleno",
    "Tonelaje vs Número de Viajes",
    "Número de Paladas vs Ciclo Total de Pala",
    "Ciclo Total de Camión vs Distancia Vacío"
])

# Asignar colores a cada gráfico de dispersión
# Tonelaje vs Número de Paladas
fig.add_trace(go.Scatter(
    x=cluster_summary_reset[('ton', 'mean')],
    y=cluster_summary_reset[('n_shovel', 'mean')],
    mode='markers',
    marker=dict(color=colors),
    text=cluster_summary_reset['cluster'],  # Agregamos el nombre del cluster como etiqueta
    name="Tonelaje vs Número de Paladas"
), row=1, col=1)

# Tonelaje vs Ciclo Total de Camión
fig.add_trace(go.Scatter(
    x=cluster_summary_reset[('ton', 'mean')],
    y=cluster_summary_reset[('truck_total_cycle', 'mean')],
    mode='markers',
    marker=dict(color=colors),
    text=cluster_summary_reset['cluster'],
    name="Tonelaje vs Ciclo Total de Camión"
), row=1, col=2)

# Tonelaje vs Distancia Recorrida Lleno
fig.add_trace(go.Scatter(
    x=cluster_summary_reset[('ton', 'mean')],
    y=cluster_summary_reset[('distance_full', 'mean')],
    mode='markers',
    marker=dict(color=colors),
    text=cluster_summary_reset['cluster'],
    name="Tonelaje vs Distancia Recorrida Lleno"
), row=1, col=3)

# Tonelaje vs Número de Viajes
fig.add_trace(go.Scatter(
    x=cluster_summary_reset[('ton', 'mean')],
    y=cluster_summary_reset[('trips', 'mean')],
    mode='markers',
    marker=dict(color=colors),
    text=cluster_summary_reset['cluster'],
    name="Tonelaje vs Número de Viajes"
), row=2, col=1)

# Número de Paladas vs Ciclo Total de Pala
fig.add_trace(go.Scatter(
    x=cluster_summary_reset[('n_shovel', 'mean')],
    y=cluster_summary_reset[('loader_total_cycle', 'mean')],
    mode='markers',
    marker=dict(color=colors),
    text=cluster_summary_reset['cluster'],
    name="Número de Paladas vs Ciclo Total de Pala"
), row=2, col=2)

# Ciclo Total de Camión vs Distancia Vacío
fig.add_trace(go.Scatter(
    x=cluster_summary_reset[('truck_total_cycle', 'mean')],
    y=cluster_summary_reset[('distance_empty', 'mean')],
    mode='markers',
    marker=dict(color=colors),
    text=cluster_summary_reset['cluster'],
    name="Ciclo Total de Camión vs Distancia Vacío"
), row=2, col=3)

# Configurar títulos y layout
fig.update_layout(
    title="Características de los Clústeres",
    showlegend=False,
    height=800,
    width=1200
)

# Mostrar gráfico interactivo
fig.show()


`Tonelaje vs. Número de Paladas`: Ilustra la relación entre el tonelaje promedio y el número de paladas, mostrando cómo los clústeres de mayor tonelaje tienden a requerir más paladas.

`Tonelaje vs. Ciclo Total de Camión`: Indica cómo el tonelaje se relaciona con el ciclo promedio del camión; los clústeres de mayor tonelaje generalmente tienen ciclos más largos.

`Tonelaje vs. Distancia Recorrida Lleno`: Muestra que las operaciones de mayor tonelaje también implican mayores distancias recorridas con carga.

`Tonelaje vs. Número de Viajes`: Los clústeres más eficientes en tonelaje no necesariamente realizan más viajes, sino que optimizan el tonelaje por cada uno.

`Número de Paladas vs. Ciclo Total de Pala`: Relación entre paladas y ciclos de pala, donde una mayor cantidad de paladas parece extender los ciclos.

`Ciclo Total de Camión vs. Distancia Recorrida Vacío`: Visualiza cómo los ciclos de los camiones se relacionan con la distancia recorrida sin carga.